In [ ]:
import os
import pandas as pd
from datetime import datetime

from config import ZONING_DATA_PATH

enter_leave_pairs = pd.read_csv(ZONING_DATA_PATH)

def sort_videos_chronologically(video_names):
    def extract_timestamp(video_name):
        timestamp_str = video_name[4:-10]
        return datetime.strptime(timestamp_str, "%Y-%m-%d-%H-%M-%S")
    
    return sorted(video_names, key=extract_timestamp)

def get_timestamp_from_filename(filename):
    timestamp_str = filename.split(".")[0][4:23]
    try:
        return datetime.strptime(timestamp_str, "%Y-%m-%d-%H-%M-%S")
    except ValueError:
        return None
    
def get_vids_between(enter_vid_path, leave_vid_path):
    enter_day_folder = os.path.dirname(enter_vid_path) + "/"
    leave_day_folder = os.path.dirname(leave_vid_path) + "/"
    
    enter_vid = enter_vid_path.split("/")[-1]
    leave_vid = leave_vid_path.split("/")[-1]
    
    enter_timestamp = get_timestamp_from_filename(enter_vid)
    leave_timestamp = get_timestamp_from_filename(leave_vid)
    
    vids_between = []
    
    for filename in os.listdir(enter_day_folder):
        timestamp = get_timestamp_from_filename(filename)
        if timestamp is not None and enter_timestamp < timestamp < leave_timestamp and filename in enter_leave_pairs["video"].values:
            vids_between.append(filename)
    if leave_day_folder != enter_day_folder:
        for filename in os.listdir(leave_day_folder):
            timestamp = get_timestamp_from_filename(filename)
            if timestamp is not None and enter_timestamp < timestamp < leave_timestamp and filename in enter_leave_pairs["video"].values:
                vids_between.append(filename)
    return sort_videos_chronologically(vids_between)

In [2]:
import numpy as np
import cv2
from pathlib import Path
import shutil

def extract_frames(video_path: str):
    cap = cv2.VideoCapture(video_path)
    frames = []
    if not cap.isOpened():
        raise ValueError(f"Could not open video file: {video_path}")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    return np.array(frames)

def is_in_zone(x:int, y:int, zone:np.ndarray) -> bool:
    return cv2.pointPolygonTest(zone, (int(x), int(y)), measureDist=False) >= 0

def is_in_zone_vec(centers:np.ndarray, zone:np.ndarray) -> np.ndarray:
    return np.array([
        is_in_zone(x, y, zone)
        for x, y in centers
    ]).astype(bool)

parking_region_pix = np.array([
    [545, 129],
    [674, 156],
    [774, 291],
    [610, 263],
])
driving_region_pix = np.array([
    [357, 60],
    [522, 85],
    [666, 385],
    [401, 385],
])

def take_drive_pics(record, vid_path, drive_pics_folder_path):
    os.makedirs(drive_pics_folder_path, exist_ok=True)
    
    record = record.copy()
    record['in_driving_region'] = is_in_zone_vec(record[['cx', 'cy']].values.astype(int), driving_region_pix)
    frames = extract_frames(vid_path)
    for i, row in record.iterrows():
        if row['in_driving_region']:
            frame = frames[row['iteration']]
            if not np.isnan(row["tl_y"]) and not np.isnan(row["tl_x"]) and not np.isnan(row["h"]) and not np.isnan(row["w"]):
                cropped = frame[int(row["tl_y"]):(int(row["tl_y"]) + int(row["h"])),
                                int(row["tl_x"]):(int(row["tl_x"]) + int(row["w"]))]
                cv2.imwrite(drive_pics_folder_path + f"frame{i}.jpg", cropped)

def create_zone(license_plate, track_id, enter_vid_path):
    zone = pd.DataFrame(columns=["license_plate", "drive_pics_folder_path"])
    date_time = str(get_timestamp_from_filename(os.path.basename(enter_vid_path)))
    drive_pics_folder_path = f"first_zone/{date_time}/{license_plate}/"
    
    record_path = enter_vid_path.replace(".mp4", ".csv")
    record_path = record_path.replace("MP4s", "CSVs")
    record = pd.read_csv(record_path)
    rel_track_record = record[record["track_id"] == track_id]
    take_drive_pics(rel_track_record, enter_vid_path, drive_pics_folder_path)

    new_car = pd.DataFrame({'license_plate': license_plate, 'drive_pics_folder_path': drive_pics_folder_path}, index=[0])
    zone = pd.concat([zone, new_car], ignore_index=True)
    return zone

def add_car_to_zone(zone, license_plate, track_id, enter_vid_path):
    date_time = str(get_timestamp_from_filename(os.path.basename(enter_vid_path)))
    drive_pics_folder_path = f"first_zone/{date_time}/{license_plate}/"
    
    record_path = enter_vid_path.replace(".mp4", ".csv")
    record_path = record_path.replace("MP4s", "CSVs")
    record = pd.read_csv(record_path)
    rel_track_record = record[record["track_id"] == track_id]
    take_drive_pics(rel_track_record, enter_vid_path, drive_pics_folder_path)
    
    new_car = pd.DataFrame({'license_plate': license_plate, 'drive_pics_folder_path': drive_pics_folder_path}, index=[0])
    zone = pd.concat([zone, new_car], ignore_index=True)
    return zone

def remove_car_from_zone(zone, predicted_license_plate):
    folder_path = Path("./") / zone.loc[zone["license_plate"] == predicted_license_plate, "drive_pics_folder_path"].values[0]
    path_str = str(folder_path)
    path_str = path_str.rsplit("/", 1)[0] + "/"
    folder_path = Path(path_str)
    if folder_path.exists() and folder_path.is_dir():
        shutil.rmtree(folder_path)
    zone = zone[zone["license_plate"] != predicted_license_plate]
    return zone

In [3]:
import pandas as pd

data = pd.read_csv("../zoning_data/video_information.csv")

num_vids_between = []
for index, row in data.iterrows():
    track_id = row["track_id"]
    enter_name = None
    leave_name = None
    if row["action"] == "enter":
        enter_name = row["video"]
        leave_name = row["leave_video"]
    else:
        leave_name = row["video"]
        enter_name = row["enter_video"]

    entering_vid_path = f"../zoning_data/{enter_name[0:14]}/MP4s/{enter_name}"
    leaving_vid_path = f"../zoning_data/{leave_name[0:14]}/MP4s/{leave_name}"
    
    vids_between = get_vids_between(entering_vid_path, leaving_vid_path)
    if row["action"] == "enter" and len(vids_between) > 5 and entering_vid_path != "../zoning_data/rec-2024-12-20/MP4s/rec-2024-12-20-17-36-55_small.mp4":
        
        # create zone and put the main enter/leave car in the zone
        zone = create_zone(row["fake_license_plate"], row["track_id"], entering_vid_path)
        
        # look for each vid_between in the data, if in the data, add any enters and predict on any leaves (only if the leaving car is in the zone)
        for vid in vids_between:
            if vid in data["video"].values:
                car_data = data.loc[data["video"] == vid]
                if car_data["action"].values[0] == "enter":
                    print(f"car {car_data["fake_license_plate"].values[0]} entered")
                    vid_path_string = f"../zoning_data/{vid[0:14]}/MP4s/{vid}"
                    zone = add_car_to_zone(zone, car_data["fake_license_plate"].values[0], car_data["track_id"].values[0], vid_path_string)
                    print(zone, "\n")
                else:
                    print(f"car {car_data["fake_license_plate"].values[0]} left")
                    # predict car leaving
                    # predicted_license_plate = predict_leaving(...)
                    predicted_license_plate = car_data["fake_license_plate"].values[0]
                    if predicted_license_plate in zone["license_plate"].values:
                        zone = remove_car_from_zone(zone, predicted_license_plate)
                    else:
                        print("car not in zone to begin with")
                    print(zone, "\n")
        break
        
    num_vids_between.append(len(vids_between))
    

car IJB-4167 left
car not in zone to begin with
  license_plate                    drive_pics_folder_path
0      AOY-6401  first_zone/2024-12-27 17:42:56/AOY-6401/ 

car NVF-2877 entered
  license_plate                    drive_pics_folder_path
0      AOY-6401  first_zone/2024-12-27 17:42:56/AOY-6401/
1      NVF-2877  first_zone/2024-12-27 17:53:17/NVF-2877/ 

car NVF-2877 left
  license_plate                    drive_pics_folder_path
0      AOY-6401  first_zone/2024-12-27 17:42:56/AOY-6401/ 

car MQJ-1472 entered
  license_plate                    drive_pics_folder_path
0      AOY-6401  first_zone/2024-12-27 17:42:56/AOY-6401/
1      MQJ-1472  first_zone/2024-12-27 18:01:44/MQJ-1472/ 

car MQJ-1472 left
  license_plate                    drive_pics_folder_path
0      AOY-6401  first_zone/2024-12-27 17:42:56/AOY-6401/ 

car OUV-7286 entered
  license_plate                    drive_pics_folder_path
0      AOY-6401  first_zone/2024-12-27 17:42:56/AOY-6401/
1      OUV-7286  first_zone/202

In [65]:
import numpy as np

print(np.mean(num_vids_between))

1.825962910128388


In [75]:
print(num_vids_between)
print(np.max(num_vids_between))
print(np.sum(np.array(num_vids_between) > 9))

[0, 1, 2, 1, 1, 0, 1, 0, 4, 4, 2, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 3, 1, 9, 1, 0, 0, 0, 0, 0, 18, 1, 2, 1, 3, 3, 0, 1, 0, 0, 1, 4, 0, 0, 0, 5, 0, 1, 1, 1, 11, 2, 0, 0, 0, 4, 5, 5, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 3, 2, 0, 2, 1, 1, 0, 2, 0, 0, 0, 0, 0, 4, 0, 21, 2, 2, 5, 3, 2, 0, 1, 0, 2, 0, 1, 1, 1, 0, 0, 0, 0, 8, 0, 0, 1, 1, 0, 6, 1, 1, 2, 0, 2, 4, 0, 0, 0, 4, 1, 6, 0, 3, 0, 0, 8, 0, 0, 0, 2, 0, 0, 0, 1, 0, 3, 0, 3, 0, 2, 3, 1, 0, 1, 1, 5, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 1, 2, 1, 2, 1, 0, 0, 0, 1, 0, 3, 1, 1, 1, 4, 0, 0, 4, 1, 0, 3, 2, 0, 4, 1, 1, 0, 5, 0, 0, 0, 1, 1, 0, 1, 1, 11, 0, 6, 2, 2, 9, 3, 0, 0, 0, 6, 1, 1, 4, 0, 1, 2, 0, 2, 0, 0, 0, 0, 3, 2, 3, 3, 0, 3, 1, 8, 0, 0, 0, 0, 5, 0, 0, 1, 0, 6, 4, 1, 7, 1, 0, 35, 0, 3, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 2, 1, 0, 3, 2, 2, 0, 3, 20, 1, 7, 4, 4, 0, 4, 0, 1, 7, 2, 0, 4, 2, 2, 15, 1, 0, 0, 7, 0, 0, 0, 0, 0, 6, 0, 11, 0, 0, 0, 0, 8, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 3, 1, 4, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 4, 1, 2, 0, 4, 5